In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.datasets import load_digits
from torchvision.datasets import FashionMNIST
from torch.utils.data import Subset, DataLoader
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
import math
from torchvision import transforms
from torch.linalg import multi_dot
import gc

# Init



In [2]:

# Controlla la disponibilità della GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Imposta il dispositivo sulla GPU
else:
    device = torch.device("cpu")  # Se la GPU non è disponibile, utilizza la CPU



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#ink originale: https://drive.google.com/file/d/0B7EVK8r0v71pZjFTYXZWM3FlRnM/view?usp=drive_link&resourcekey=0-dYn9z10tMJOBAkviAcfdyQ
#spostatelo nel vostro drive e copiatelo in locale (perchè il dataloader carica più velocemente le immagini leggendo da qui che dal vostro drive)
!cp '/content/drive/MyDrive/Generative_AI/datasets/celebA/img_align_celeba.zip' celebA.zip

In [5]:
import zipfile

with zipfile.ZipFile("celebA.zip", 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

#Model

In [6]:
#Gaussian Error Linear Unit (gelus)
class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return x*torch.sigmoid(1.702*x)

In [14]:
class HierarchicalVAE(nn.Module):
  def __init__(self, input_shape_image, latent_space_dimension, hidden_neurons,possible_pixel_values):
    super(HierarchicalVAE, self).__init__()

    self.input_shape_image = input_shape_image #es. (8,8) immagine -> 64
    self.latent_space_dimension = latent_space_dimension
    self.hidden_neurons = hidden_neurons
    self.possible_pixel_values = possible_pixel_values


    self.r1_net = nn.Sequential(
                                nn.Linear(input_shape_image, 2*hidden_neurons),
                                GELU(),
                                nn.BatchNorm1d(2*hidden_neurons),
                                nn.Linear(2*hidden_neurons, hidden_neurons),
                                GELU())

    self.r2_net = nn.Sequential(
                                nn.Linear(hidden_neurons, 2*hidden_neurons),
                                GELU(),
                                nn.BatchNorm1d(2*hidden_neurons),
                                nn.Linear(2*hidden_neurons, hidden_neurons),
                                nn.LeakyReLU())

    #rete che genera i delta di media e varianza per la variabile z2
    #avendo solo due variabili latenti, non avremo una delta per z2, ma direttamente la rete
    #che genera la sua media e varianza
    self.net_z2 = nn.Sequential(
                            nn.Linear(hidden_neurons, 2*hidden_neurons),
                            GELU(),
                            nn.BatchNorm1d(2*hidden_neurons),
                            #produco media e log std per z2
                            nn.Linear(2*hidden_neurons, 2*latent_space_dimension),
                            nn.Hardtanh(-7,2)
                            )

    ##rete che genera media e varianza per la variabile z1
    self.net_z1 = nn.Sequential(
                            nn.Linear(latent_space_dimension, 2*hidden_neurons),
                            GELU(),
                            nn.BatchNorm1d(2*hidden_neurons),
                            #produco media e log std per z1
                            nn.Linear(2*hidden_neurons, 4*latent_space_dimension),
                            nn.Hardtanh(-7,2)
                            )

    #rete che genera i delta di media e varianza per la variabile z1
    self.net_delta_z1 = nn.Sequential(
                                nn.Linear(hidden_neurons, 2*hidden_neurons),
                                GELU(),
                                nn.BatchNorm1d(2*hidden_neurons),
                                #produco delta per media e log std di z1
                                nn.Linear(2*hidden_neurons, 4*latent_space_dimension),
                                )

    self.net_reconstruction = nn.Sequential(
                                nn.Linear(2*latent_space_dimension, hidden_neurons),
                                GELU(),
                                nn.BatchNorm1d(hidden_neurons),
                                nn.Linear(hidden_neurons,2*hidden_neurons),
                                GELU(),
                                nn.BatchNorm1d(2*hidden_neurons),
                                #produco media e log std per z1
                                nn.Linear(2*hidden_neurons, input_shape_image*possible_pixel_values)
                                )


  def sample(self):
    #creo la prior p(z2)=N(z|0,I), è sempre la stessa
    p_z2 = MultivariateNormal(torch.zeros(self.latent_space_dimension).to(device), torch.eye(self.latent_space_dimension).to(device))
    z2_sample = p_z2.sample()
    z2_sample = z2_sample.unsqueeze(0)

    #inietto nella net_z1
    #utilizzo z2 per ottenere i parametri della distribuzione di z1
    mean_and_log_std_z1 = self.net_z1(z2_sample)
    mean_z1, log_std_z1 = torch.chunk(mean_and_log_std_z1, 2, dim=1)


    #creo la distribuzione da cui campionare z1 e campiono z1
    z1_distribution = MultivariateNormal(mean_z1, torch.diag_embed( torch.clamp( torch.exp(log_std_z1),min=1e-8)))
    z1_sample = z1_distribution.sample()

    #inietto z1 nella rete di ricostruzione
    output = self.net_reconstruction(z1_sample)

    #reshape in (1,input_shape, possible_pixel_values)
    logits = output.reshape(1,self.input_shape_image, self.possible_pixel_values)
    probabilities = torch.softmax(logits, dim=-1)
    probabilities = probabilities.view(-1, self.possible_pixel_values)


    sample = torch.multinomial(probabilities, num_samples=1)

    x = sample.view(self.input_shape_image)
    return x


  def forward(self, x):

    #reshape da (N, W, H) a (N,W*H)
    x = torch.flatten(x,1,2)

    #normalizzo il batch
    x_norm = x/(self.possible_pixel_values-1)

    #elaboro x per ottenere R1
    r1 = self.r1_net(x_norm)

    #elaboro r1 per ottenere R2
    r2 = self.r2_net(r1)

    #elaboro R1 per creare i delta medie e log std da aggiungere a z1
    delta_z1 = self.net_delta_z1(r1)
    #splitto in due delta, uno per media e uno per log std
    delta_mean_z1, delta_log_std_z1 = torch.chunk(delta_z1, 2, dim=1)

    #elaboro R2 per trovarmi i parametri della distribuzione di z2
    mean_and_log_std_z2 = self.net_z2(r2)
    #splitto in media e log std
    mean_z2, log_std_z2 = torch.chunk(mean_and_log_std_z2, 2, dim=1)

    #campiono dalla distribuzione di z2 per ottenere z2 (con reparametrization trick)
    z2_distribution = MultivariateNormal(mean_z2, torch.diag_embed( torch.clamp( torch.exp(log_std_z2),min=1e-8)))
    z2 = z2_distribution.rsample()

    #utilizzo z2 per ottenere i parametri della distribuzione di z1
    mean_and_log_std_z1 = self.net_z1(z2)
    mean_z1, log_std_z1 = torch.chunk(mean_and_log_std_z1, 2, dim=1)

    #campiono z1 (correggendo la media e la log std con i delta)
    z1_distribution = MultivariateNormal(mean_z1 +  delta_mean_z1, torch.diag_embed( torch.clamp( torch.exp(log_std_z1*delta_log_std_z1),min=1e-8)))
    z1 = z1_distribution.rsample()

    #utilizzo la rete per ricostruire l'immagine (N, input_shape*possible_pixel_values)
    output = self.net_reconstruction(z1)

    #reshape in (N, input_shape, possible_pixel_values)
    logits = output.reshape(output.shape[0],self.input_shape_image, self.possible_pixel_values)

    #eseguo la softmax sull'ultimo livello
    output_probabilities = torch.softmax(logits,dim=-1)
    EPS = 1.e-5
    output_probabilities = torch.clamp(output_probabilities, EPS,1. - EPS)
    #li converto in logaritmi
    log_probabilities = torch.log(output_probabilities)

    #trasformo x in one hot encoding
    x_one_hot = F.one_hot(x.long(),num_classes=self.possible_pixel_values)

    #li utilizzo per azzerare la probabilità relativa al valore del pixel
    selected_probabilities = x_one_hot*log_probabilities

    #li sommo (N,), ossia ogni cella contiene la somma dei ln(p(x|z1)) per gli N x
    #Questo sarebbe il Reconstruction Error
    RE = selected_probabilities.sum(-1).sum(-1)



    '''
      Per calcolare il KL error so dalla teoria che:
              KL = E[ln(q(z1|x)/(p(z1|z2))) + ln(q(z2|z1)/p(z2))]
      Avendo cambiato le dipendenze per unire il cuore dell'encoder con quello
      del decoder, allora:

              KL = E[ln(q(z1|z2)/(p(z1|z2))) + ln(q(z2|x)/p(z2))] = KL1 + KL2

      dove:
      - q(z1|z2) sarebbe la z1_distribution
      - q(z2|x)  sarebbe la z2_distribution
      - p(z1|z2) sarebbe la N(u,var) dove u e var sono quelli che ha prodotto la net_z1
      - p(z2) sarebbe la N(0,I)

      (NB: non sto eseguendo alcun Monte-Carlo per approssimare l'expected value)
      Inoltre essendo p e q gaussiane esiste una formula chiusa:
                KLi = [delta_mean^2/var^2 + delta_var^2 - ln(delta_var^2) - 1]
      NB: il "quadrato" per le varianze è da mettere se le distribuzioni sono state
      create "passando" la varianza quadra (in quanto quando do alla Multivariate la covariance
      matrix questa la intende con elementi nella diagonale al quadrato); noi abbiamo passato
      solo le varianze (niente quadrato) e quindi saranno da intendere già al quadrato.
    '''
    #SE LE PRIOR NON SONO GAUSSIANE
    # p_z2 = MultivariateNormal(torch.zeros(z2.shape[1]).to(device), torch.eye(z2.shape[1]).to(device))
    # p_z1_z2 = MultivariateNormal(mean_z1, torch.diag_embed( torch.clamp( torch.exp(log_std_z1),min=1e-8)))

    # KL_z1 = z1_distribution.log_prob(z1) - p_z1_z2.log_prob(z1)

    # KL_z2 = z2_distribution.log_prob(z2) - p_z2.log_prob(z2)

    #SE LE PRIOR SONO GAUSSIANE ESISTE UNA FORMULA CHIUSA
    KL_z2 = 0.5 * (mean_z2**2 + torch.exp(log_std_z2) - log_std_z2 - 1).sum(-1)
    KL_z1 = 0.5 * (mean_z1**2 / torch.exp(log_std_z1) + torch.exp(log_std_z1) -log_std_z1 - 1).sum(-1)

    KL = KL_z1 + KL_z2

    #ELBO (medio l'ELBO di ogni immagine)
    ELBO = (KL-RE).mean()

    return ELBO





# MAIN

In [ ]:
##################################### GPU + Path ##########################################

# Controlla la disponibilità della GPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # Imposta il dispositivo sulla GPU
else:
    device = torch.device("cpu")  # Se la GPU non è disponibile, utilizza la CPU

print("Device utilizzato:", device)
print("Numero di GPU disponibili:", torch.cuda.device_count())


#path dove salvare il modello migliore e i vari output di ogni epoca valida
path_to_model = "/content/drive/MyDrive/Generative_AI/datasets/celebA/model/model_Hierarchical_VAE.pth"
path_to_output = "/content/drive/MyDrive/Generative_AI/datasets/celebA/output/Hierarchical_VAE_"





##################################### Dataloader ##########################################

#per motivi di efficienza, scegliere il rescaling e il massimo valore che ogni pixel può assumere
resize_to = 64
max_pixel_value = 20

input_shape_image = resize_to*resize_to
possible_pixel_values = max_pixel_value+1


def load_data():


    # Definisci le trasformazioni da applicare alle immagini durante il caricamento
    transform = transforms.Compose([
        transforms.Resize( (resize_to, resize_to) ), #rescaling di ogni immagine
        transforms.Grayscale(),  # Trasforma l'immagine in bianco e nero
        transforms.ToTensor(),# Converte l'immagine in un tensore
        transforms.Lambda(lambda x: torch.round(x*(max_pixel_value))), #normalizzo i valori dei pixel e li forzo ad essere interi
        #transforms.Lambda(lambda x: x.to(torch.float32))
    ])

    # Crea un oggetto ImageFolder per caricare le immagini dalla cartella specificata e applica le trasformazioni definite
    dataset = datasets.ImageFolder('/content/dataset/', transform=transform)

    # Calcola l'indice per dividere il dataset tra training set e validation set
    split_ratio = 0.8  # Ratio di suddivisione (80% per il training set, 20% per il validation set)
    dataset_size = len(dataset)
    split_index = int(split_ratio * dataset_size)

    # Crea due sottoinsiemi distinti per il training set e il validation set
    train_dataset = Subset(dataset, range(0, split_index))
    val_dataset = Subset(dataset, range(split_index, dataset_size))

    # Crea i DataLoader per il training set e il validation set
    batch_size = 32
    training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return training_loader, validation_loader










################################### Training + validation #####################################

def train_model_on_given_gpu():

    #definisco la dimensione dello spazio latente
    latent_space_dimension = 32 #deve essere pari se utilizzi il Flow-based
    #nuumero di hidden neurons nell'encoder e decoder
    hidden_neurons = 64


    #---- creazione del modello

    model = HierarchicalVAE(input_shape_image, latent_space_dimension, hidden_neurons,possible_pixel_values)
    model.to(device)

    print("Numero parametri modello: ",sum(p.numel() for p in model.parameters() if p.requires_grad))

    #parametri per il learning
    learning_rate = 1e-3

    parameters_to_optimize = [p for p in model.parameters() if p.requires_grad == True]

    optimizer = torch.optim.Adamax(parameters_to_optimize, lr=learning_rate)


    #------ Funzione per salvare una griglia di campioni decodificati dallo spazio latente ogni volta che la validation è migliore
    def sample_and_save(model, name, input_shape):

      model.eval()

      #voglio campionare 16 immagini e le voglio in una griglia 4x4
      n=4
      number_of_grid_cells = n*n
      #quindi dico al modello di campionarmi 16 immagini
      xs = np.zeros((number_of_grid_cells,input_shape))
      for i in np.arange(number_of_grid_cells):
        generated_sample = model.sample().cpu() # il .module serve per andare oltre il wrapping di DataParallel
        #lo stacco dal grafo di computazione
        generated_sample = generated_sample.detach().numpy()
        xs[i,:] = generated_sample


      fig, ax = plt.subplots(n, n)
      for i, ax in enumerate(ax.flatten()):
          plottable_image = np.reshape(xs[i], (int(math.sqrt(input_shape)), int(math.sqrt(input_shape))))
          ax.imshow(plottable_image, cmap='gray')
          ax.axis('off')

      plt.savefig(path_to_output+'epoca_' +str(name)+ '.pdf', bbox_inches='tight')
      plt.close()



    #---- Training e validation
    number_of_epochs = 1000
    #fisso il limite massimo di batch di training e validazione
    max_batch_for_training = 800
    max_batch_for_validation = 170


    #qui salvo il migliore modello, ossia quello che ha la loss sulla validazione migliore
    best_model = model
    best_validation_loss = 1000000

    patience = 0
    max_patience = 30

    training_loader, validation_loader = load_data()

    grd_acc = 1 #significa che prima di backpropagare l'errore accumulerò il gradiente di batch_size*grd_acc (ees. 8*4=32 è come se processassi batch da 32)

    for epoch in range(number_of_epochs):
      model.train()
      print("Epoca "+str(epoch)+" _____________________________________________________________________")

      num_batch = 1
      for batch, _ in training_loader:

        #reshaping di ogni batch da (N, 1, W, H) a (N, W, H)
        batch = batch.squeeze(1)

        batch = batch.to(device)
        #print("             Memoria GPU utilizzata prima loss per batch:",num_batch,"  -> ", round(torch.cuda.memory_allocated()*(1e-9),5)," GB")
        #batch = batch.to(torch.float32)

        loss = model.forward(batch)
        #print("             Memoria GPU utilizzata dopo loss per batch:",num_batch,"  -> ", round(torch.cuda.memory_allocated()*(1e-9),5)," GB")
        del batch
        gc.collect()
        torch.cuda.empty_cache()

        #calcolo le derivate parziali della loss rispetto ogni parametro NB: LA mean() E' PERCHE' UTILIZZO N GPU E CIASCUNA RITORNA LA SUA LOSS
        (loss.mean()/grd_acc).backward(retain_graph=True)
        torch.cuda.empty_cache()
        #print("             Memoria GPU utilizzata dopo backward per batch:",num_batch,"  -> ", round(torch.cuda.memory_allocated()*(1e-9),5)," GB")
        #se ho accumulato il gradiente di un numero sufficiente di batch, allora backpropago
        if ( (num_batch % grd_acc) == 0):
            #adesso ogni parametro ha in .grad il gradiente. Aggiorno il suo valore
            optimizer.step()

            #resetto il .grad di ogni parametro (altrimenti sommo quello attuale al successivo che calcoleremo nell'epoca dopo)
            optimizer.zero_grad()

        print("   Loss batch: ",str(num_batch),": ", loss, "          Memoria GPU utilizzata  -> ", round(torch.cuda.memory_allocated()*(1e-9),4), "GB")

            #se ho superato il numero massimo di batch per il training, esco
        if num_batch >= max_batch_for_training:
            break
        else:
            num_batch = num_batch + 1

      #alla fine di ogni epoca, valuto come si comporta la loss col validation set
      print("   ___________________________")
      model.eval()
      validation_loss = 0
      N = 0

      torch.cuda.empty_cache()

      num_batch = 1
      for batch, _ in validation_loader:

        batch = batch.squeeze(1)
        batch = batch.to(device)
        #batch = batch.to(torch.float32)
        loss_i = model.forward(batch)
        validation_loss = validation_loss + loss_i.mean().item()# NB: .mean() SOLO PERCH' UTILIZZO N GPU E QUINDI VOGLIO LA MEDIA DI OGNI LOSS RITORNATA DA OGNI GPU
        N = N +  1

        del batch
        gc.collect()
        torch.cuda.empty_cache()

        print("   Loss validation batch ",str(num_batch),": ",loss_i)
        #se ho superato il numero massimo di batch per il validation, esco
        if num_batch >= max_batch_for_validation:
            break
        else:
            num_batch = num_batch + 1

        del loss_i

      validation_loss = validation_loss/N
      print("   Loss media validation: ",str(validation_loss))

      #se tale modello ha una loss migliore di quella attualmente migliore..
      if validation_loss < best_validation_loss:
        patience = 0
        best_validation_loss = validation_loss
        print("   la loss risulta essere migliore")
        torch.save(model.state_dict(), path_to_model)
        #campiono e salvo
        sample_and_save(model, epoch, input_shape_image)
      else:
        print("   patience= "+ str(patience+1))
        patience = patience + 1

      if patience > max_patience:
        print("")
        print("Patience massimo superato. Fine del training")
        break

      del validation_loss



if __name__=="__main__":

    train_model_on_given_gpu()